# Applying KNN to data

Here we will be using K Nearest Neighbors to help with the imputation of our missing data

In [39]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.impute import KNNImputer


np.random.seed(42)

import missingno as msno

## Establishing our save/extract path

In [40]:
path = "../SHARED/"

## KNN for Recompletion data

In [82]:
# Read in the data and only work with the data that has a valid Contractor
data_pd =  pd.read_excel(path + "TO_TACC_RCOMPL.xlsx")
dataframe = data_pd[data_pd['CONTRACTOR'].notnull()]

In [83]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1654 entries, 0 to 1653
Data columns (total 79 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   #WELL_NAME                     1654 non-null   object        
 1   CURRENT_STATUS                 1654 non-null   object        
 2   UWI                            1654 non-null   int64         
 3   UWI_SIDETRACK                  1654 non-null   int64         
 4   JOB_CATEGORY                   1654 non-null   object        
 5   JOB_START_DATE                 1654 non-null   object        
 6   JOB_END_DATE                   689 non-null    object        
 7   JOB_STATUS                     908 non-null    object        
 8   LIKE_KIND                      1569 non-null   object        
 9   PROJECT_TYPE                   1654 non-null   object        
 10  PHASE_START_DATE               1635 non-null   object        
 11  PHASE_END_DATE   

In [84]:
# Considering how KNN requires purely numerical data, we will need to split all the 
# Dates into their respective year, month, day, and hour.

# So we first do this by converting all dates into pd.datetime objects to
# work with them easier
dates = ['JOB_START_DATE', 'JOB_END_DATE', 'PHASE_START_DATE', 'PHASE_END_DATE', 'REPORT_START_DATE', 'REPORT_END_DATE', 'STIMULATION_TREAT_DATE']
for d in dates:
    dataframe[d] = pd.to_datetime(dataframe[d], dayfirst=True)

In [85]:
# Collect datetime objects
temp = dataframe.loc[:, dataframe.dtypes=='datetime64[ns]']

In [86]:
temp

,JOB_START_DATE,JOB_END_DATE,PHASE_START_DATE,PHASE_END_DATE,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,REPORT_START_DATE,REPORT_END_DATE,STIMULATION_TREAT_DATE,STG_START_DATE,STG_END_DATE
0,2018-12-16 06:00:00,2019-12-10 10:00:00,2019-11-01 06:00:00,2019-06-02 07:00:00,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 06:00:00,2019-01-29 06:00:00,2019-01-28 08:05:00,2019-01-28 08:05:00,2019-01-28 10:05:00
1,2018-01-21 10:00:00,2018-09-10 07:00:00,2018-01-27 06:00:00,2018-02-23 18:00:00,2018-02-14 08:00:00,2018-02-23 18:00:00,2018-02-19 06:00:00,2018-02-20 06:00:00,2018-02-19 14:40:00,2018-02-19 14:40:00,2018-02-20 00:19:00
2,2017-08-31 13:00:00,NaT,2017-07-09 00:00:00,2017-10-29 17:00:00,2017-10-25 06:00:00,2017-10-29 06:00:00,2017-10-26 06:00:00,2017-10-27 06:00:00,2017-10-26 10:13:00,2017-10-26 10:13:00,2017-10-26 12:45:00
3,2019-02-13 07:30:00,NaT,2019-04-03 16:30:00,2019-04-14 16:00:00,2019-03-24 18:00:00,2019-03-30 06:00:00,2019-03-25 06:00:00,2019-03-26 06:00:00,2019-03-25 14:05:00,2019-03-25 14:05:00,2019-03-25 15:34:00
4,2019-04-04 07:30:00,2019-06-15 18:00:00,2019-04-23 07:00:00,2019-06-15 18:00:00,2019-06-06 06:00:00,2019-06-15 06:00:00,2019-06-13 06:00:00,2019-06-14 06:00:00,2019-06-13 07:17:00,2019-06-13 07:17:00,2019-06-13 09:19:00
...,...,...,...,...,...,...,...,...,...,...,...
1649,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-01-08 15:26:00,2019-01-08 15:26:00,2019-01-08 16:21:00
1650,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-01-08 18:21:00,2019-01-08 18:21:00,2019-01-08 19:23:00
1651,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-01-08 21:27:00,2019-01-08 21:27:00,2019-01-08 22:29:00
1652,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-02-08 00:30:00,2019-02-08 00:30:00,2019-02-08 01:31:00


In [87]:
date_features = temp.columns

In [88]:
# A simple function that helps with parsing out the necessary information
# and then putting it into a temporary dataframe

def split_date(name):
    name_y = name[:-4] + "Y"
    name_m = name[:-4] + "M"
    name_d = name[:-4] + "D"
    name_h = name[:-4] + "H"
    name_min = name[:-4] + "MIN"
    temp[name_y] = np.nan
    temp[name_m] = np.nan
    temp[name_d] = np.nan
    temp[name_h] = np.nan
    temp[name_min] = np.nan
    
    
    for i in range(0, len(temp)):
        timestamp = temp.loc[i, name]
        if not pd.isnull(timestamp) :
            temp.loc[i, name_y] = timestamp.year
            temp.loc[i, name_m] = timestamp.month
            temp.loc[i, name_d] = timestamp.day
            temp.loc[i, name_h] = timestamp.hour
            temp.loc[i, name_min] = timestamp.minute

In [89]:
# Going ahead and splitting all the dates that we are working with
for i in date_features:
    split_date(i)

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_l

In [90]:
temp

,JOB_START_DATE,JOB_END_DATE,PHASE_START_DATE,PHASE_END_DATE,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,REPORT_START_DATE,REPORT_END_DATE,STIMULATION_TREAT_DATE,STG_START_DATE,...,STG_START_Y,STG_START_M,STG_START_D,STG_START_H,STG_START_MIN,STG_END_Y,STG_END_M,STG_END_D,STG_END_H,STG_END_MIN
0,2018-12-16 06:00:00,2019-12-10 10:00:00,2019-11-01 06:00:00,2019-06-02 07:00:00,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 06:00:00,2019-01-29 06:00:00,2019-01-28 08:05:00,2019-01-28 08:05:00,...,2019.0,1.0,28.0,8.0,5.0,2019.0,1.0,28.0,10.0,5.0
1,2018-01-21 10:00:00,2018-09-10 07:00:00,2018-01-27 06:00:00,2018-02-23 18:00:00,2018-02-14 08:00:00,2018-02-23 18:00:00,2018-02-19 06:00:00,2018-02-20 06:00:00,2018-02-19 14:40:00,2018-02-19 14:40:00,...,2018.0,2.0,19.0,14.0,40.0,2018.0,2.0,20.0,0.0,19.0
2,2017-08-31 13:00:00,NaT,2017-07-09 00:00:00,2017-10-29 17:00:00,2017-10-25 06:00:00,2017-10-29 06:00:00,2017-10-26 06:00:00,2017-10-27 06:00:00,2017-10-26 10:13:00,2017-10-26 10:13:00,...,2017.0,10.0,26.0,10.0,13.0,2017.0,10.0,26.0,12.0,45.0
3,2019-02-13 07:30:00,NaT,2019-04-03 16:30:00,2019-04-14 16:00:00,2019-03-24 18:00:00,2019-03-30 06:00:00,2019-03-25 06:00:00,2019-03-26 06:00:00,2019-03-25 14:05:00,2019-03-25 14:05:00,...,2019.0,3.0,25.0,14.0,5.0,2019.0,3.0,25.0,15.0,34.0
4,2019-04-04 07:30:00,2019-06-15 18:00:00,2019-04-23 07:00:00,2019-06-15 18:00:00,2019-06-06 06:00:00,2019-06-15 06:00:00,2019-06-13 06:00:00,2019-06-14 06:00:00,2019-06-13 07:17:00,2019-06-13 07:17:00,...,2019.0,6.0,13.0,7.0,17.0,2019.0,6.0,13.0,9.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-01-08 15:26:00,2019-01-08 15:26:00,...,2019.0,1.0,8.0,15.0,26.0,2019.0,1.0,8.0,16.0,21.0
1650,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-01-08 18:21:00,2019-01-08 18:21:00,...,2019.0,1.0,8.0,18.0,21.0,2019.0,1.0,8.0,19.0,23.0
1651,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-01-08 21:27:00,2019-01-08 21:27:00,...,2019.0,1.0,8.0,21.0,27.0,2019.0,1.0,8.0,22.0,29.0
1652,2019-05-31 07:00:00,NaT,2019-06-17 06:00:00,2019-02-08 18:00:00,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 06:00:00,2019-02-08 06:00:00,2019-02-08 00:30:00,2019-02-08 00:30:00,...,2019.0,2.0,8.0,0.0,30.0,2019.0,2.0,8.0,1.0,31.0


In [91]:
# Next go ahead and append that extracted date data into the original file
dataframe = pd.concat([dataframe, temp], axis = 1, sort=False)

In [92]:
# Then drop the original date columns
dataframe = dataframe.drop(date_features, axis = 1)

In [93]:
dataframe

,#WELL_NAME,CURRENT_STATUS,UWI,UWI_SIDETRACK,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,...,STG_START_Y,STG_START_M,STG_START_D,STG_START_H,STG_START_MIN,STG_END_Y,STG_END_M,STG_END_D,STG_END_H,STG_END_MIN
0,WELL - 2019000699,Producing,2019000699,0,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,1.0,28.0,8.0,5.0,2019.0,1.0,28.0,10.0,5.0
1,WELL - 2019000665,Producing,2019000665,0,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,...,2018.0,2.0,19.0,14.0,40.0,2018.0,2.0,20.0,0.0,19.0
2,WELL - 2019000347,Producing,2019000347,0,COMPLETIONS,NaN,OCM - PP PUC,RECOMPLETION,COMPZN,STIM,...,2017.0,10.0,26.0,10.0,13.0,2017.0,10.0,26.0,12.0,45.0
3,WELL - 2019001412,Producing,2019001412,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,3.0,25.0,14.0,5.0,2019.0,3.0,25.0,15.0,34.0
4,WELL - 2019001390,Producing,2019001390,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,6.0,13.0,7.0,17.0,2019.0,6.0,13.0,9.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,1.0,8.0,15.0,26.0,2019.0,1.0,8.0,16.0,21.0
1650,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,1.0,8.0,18.0,21.0,2019.0,1.0,8.0,19.0,23.0
1651,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,1.0,8.0,21.0,27.0,2019.0,1.0,8.0,22.0,29.0
1652,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,2.0,8.0,0.0,30.0,2019.0,2.0,8.0,1.0,31.0


## Label Encoding

Again, seeing how KNN requires numerical data, we need to go ahead and find a way to make all off our categorical data into numerical data, this will be done through label encoding

In [94]:
# First take all the objects (or strings in this case) to work on
all_obs = dataframe.loc[:, dataframe.dtypes==object]

In [95]:
all_obs

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,RIG_ID,RIG_TYPE,...,DIVERSION_COMPANY,DIVERSION_METHOD,DELIVERY_MODE,BOTTOM_HOLE_PRESSURE_METHOD,CLOSURE_PRESSURE_METHOD,STIMULATION_RESULT,STIMULATION_RESULT_DETAIL,STIMTREAT_ID,FLUID_NAME,SITE_SUPERVISOR
0,WELL - 2019000699,Producing,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,NaN,Success,According to Plan,A6066AE1E85F4B1DB15D9940B5FBBEBF,Slickwater,MIKE HOOSE
1,WELL - 2019000665,Producing,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000037,SILICA FLOUR,Casing,NaN,NaN,Success,According to Plan,6ACEA3AEC54E415B8251B86897A229F3,FRESH WATER,NaN
2,WELL - 2019000347,Producing,COMPLETIONS,NaN,OCM - PP PUC,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000084,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000003,Ball 40 7/8 BIO,Casing,Calculated,Nolte-Smith Calc,Success,According to Plan,29626CB5494D45ACB9C7723B028E9DEF,Slick Water,NaN
3,WELL - 2019001412,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,NaN,Success,According to Plan,DCCB82735EEA4540A1C0C402180C5A45,FRP-1S,CHRIS SANCHEZ
4,WELL - 2019001390,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,NaN,Frac Plug,Casing,NaN,NaN,Success,According to Plan,B8733A01BFFD46CC8CCDE726BD4B484B,FRP-1S,JW KRAMER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,NaN,Success,According to Plan,3378EA55ECFB4D998B852424A305E2BF,Slickwater,SEAN CHANCE
1650,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,NaN,Success,According to Plan,D59E4DC0E1654C11B392325DACBEBD11,Slickwater,NATHAN JAYROE
1651,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,NaN,Success,According to Plan,4BC64FC4AF8E463989399F926EFD55D2,Slickwater,NATHAN JAYROE
1652,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,NaN,Success,According to Plan,C464F9B9992947379161C633C1822EA9,Slickwater,NATHAN JAYROE


In [96]:
# Next apply pandas built in label encoding by exploiting their "category"
# object, so create an extra column to store these codes to then use in
# the imputation
init_columns = all_obs.columns

for c in init_columns:
    all_obs[c] = all_obs[c].astype('category')
    cat_label = c + "_cat"
    all_obs[cat_label] = all_obs[c].cat.codes
    

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [97]:
# Next because there might be some missing data in the label data, replace
# all the -1s as NaNs
for c in init_columns:
    cat_label = c + "_cat"
    all_obs[cat_label] = all_obs[cat_label].replace([-1], np.nan)

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [98]:
all_obs

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,RIG_ID,RIG_TYPE,...,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,CLOSURE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat,SITE_SUPERVISOR_cat
0,WELL - 2019000699,Producing,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1074,33.0,45.0
1,WELL - 2019000665,Producing,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,11.0,15.0,0.0,NaN,NaN,1.0,0.0,699,20.0,NaN
2,WELL - 2019000347,Producing,COMPLETIONS,NaN,OCM - PP PUC,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000084,FRAC EQUIP,...,0.0,5.0,0.0,0.0,0.0,1.0,0.0,258,32.0,NaN
3,WELL - 2019001412,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1419,24.0,11.0
4,WELL - 2019001390,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,NaN,8.0,0.0,NaN,NaN,1.0,0.0,1179,24.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,320,33.0,59.0
1650,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1375,33.0,47.0
1651,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,479,33.0,47.0
1652,WELL - 2019000628,Producing,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1252,33.0,47.0


In [99]:
# Next go ahead and drop the initial label columns to then impute
temp_df = all_obs.drop(init_columns, axis=1)

In [100]:
temp_df

,#WELL_NAME_cat,CURRENT_STATUS_cat,JOB_CATEGORY_cat,JOB_STATUS_cat,LIKE_KIND_cat,PROJECT_TYPE_cat,PHASE_cat,JOB_OP_CODE_cat,RIG_ID_cat,RIG_TYPE_cat,...,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,CLOSURE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat,SITE_SUPERVISOR_cat
0,29,0,0,0.0,1.0,0,0,0,2,1,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1074,33.0,45.0
1,21,0,0,0.0,3.0,0,0,0,5,1,...,11.0,15.0,0.0,NaN,NaN,1.0,0.0,699,20.0,NaN
2,5,0,0,NaN,2.0,0,0,0,3,1,...,0.0,5.0,0.0,0.0,0.0,1.0,0.0,258,32.0,NaN
3,65,0,0,NaN,1.0,0,0,0,5,1,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1419,24.0,11.0
4,62,0,0,NaN,1.0,0,0,0,5,1,...,NaN,8.0,0.0,NaN,NaN,1.0,0.0,1179,24.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,11,0,0,NaN,1.0,0,0,0,5,1,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,320,33.0,59.0
1650,11,0,0,NaN,1.0,0,0,0,5,1,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1375,33.0,47.0
1651,11,0,0,NaN,1.0,0,0,0,5,1,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,479,33.0,47.0
1652,11,0,0,NaN,1.0,0,0,0,5,1,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1252,33.0,47.0


In [101]:
# Now join that categorical data of the labels to the original data
data_with_labels = pd.concat([dataframe, temp_df], axis=1, sort = False)

In [102]:
data_with_labels

,#WELL_NAME,CURRENT_STATUS,UWI,UWI_SIDETRACK,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,...,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,CLOSURE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat,SITE_SUPERVISOR_cat
0,WELL - 2019000699,Producing,2019000699,0,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1074,33.0,45.0
1,WELL - 2019000665,Producing,2019000665,0,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,...,11.0,15.0,0.0,NaN,NaN,1.0,0.0,699,20.0,NaN
2,WELL - 2019000347,Producing,2019000347,0,COMPLETIONS,NaN,OCM - PP PUC,RECOMPLETION,COMPZN,STIM,...,0.0,5.0,0.0,0.0,0.0,1.0,0.0,258,32.0,NaN
3,WELL - 2019001412,Producing,2019001412,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1419,24.0,11.0
4,WELL - 2019001390,Producing,2019001390,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,NaN,8.0,0.0,NaN,NaN,1.0,0.0,1179,24.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,320,33.0,59.0
1650,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1375,33.0,47.0
1651,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,479,33.0,47.0
1652,WELL - 2019000628,Producing,2019000628,0,COMPLETIONS,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,8.0,0.0,NaN,NaN,1.0,0.0,1252,33.0,47.0


In [103]:
# Now seeing how this data still has the initial column data i.e. the labels themselves
# we need to remove them
actual_data = data_with_labels.drop(init_columns, axis=1)

In [104]:
# Now for reference we want to see how many missing values are in our initial data
count = 0
for i in actual_data.columns:
# count number of rows with missing values
    n_miss = actual_data[i].isnull().sum()
    perc = n_miss / actual_data.shape[0] * 100
    print('> %2d %28s, Missing: %d (%.1f%%)' % (count, i, n_miss, perc))
    count += 1

>  0                          UWI, Missing: 0 (0.0%)
>  1                UWI_SIDETRACK, Missing: 0 (0.0%)
>  2       REPORT_START_DATE_YEAR, Missing: 0 (0.0%)
>  3                FRAC_GRADIENT, Missing: 393 (23.8%)
>  4           BREAKDOWN_PRESSURE, Missing: 303 (18.3%)
>  5         HYDROSTATIC_PRESSURE, Missing: 43 (2.6%)
>  6           TREAT_AVG_PRESSURE, Missing: 2 (0.1%)
>  7           TREAT_MAX_PRESSURE, Missing: 2 (0.1%)
>  8           TREAT_MIN_PRESSURE, Missing: 746 (45.1%)
>  9            PROPPANT_DESIGNED, Missing: 1 (0.1%)
> 10        PROPPANT_IN_FORMATION, Missing: 0 (0.0%)
> 11         PROPPANT_IN_WELLBORE, Missing: 415 (25.1%)
> 12          AVG_HYDRAULIC_POWER, Missing: 3 (0.2%)
> 13               MAX_PUMP_POWER, Missing: 49 (3.0%)
> 14              MAX_PUMP_RATING, Missing: 260 (15.7%)
> 15               TREAT_AVG_RATE, Missing: 2 (0.1%)
> 16               TREAT_MAX_RATE, Missing: 2 (0.1%)
> 17               TREAT_MIN_RATE, Missing: 10 (0.6%)
> 18 POST_INSTANT_SHUT_IN_PR

## KNN Imputation

In [105]:
# Now to the actual imputation

# here we are going to get rid of column 109 (Contractor) and use that as our
# prediction variable
data = actual_data.values
ix = [i for i in range(data.shape[1]) if i != 109]
X, y = data[:, ix], data[:, 109]

In [106]:
# define imputer
imputer = KNNImputer()
# fit on the dataset
imputer.fit(X)

KNNImputer()

In [107]:
# Now fill in the missing data
Xtrans = imputer.transform(X)

In [108]:
# Now here is the newly imputed data
print('missing: %d' % sum(np.isnan(Xtrans).flatten()))

missing: 0


## Reintroduction into the main data

Now we need to go ahead and export these newly imputed data points into the main file along with reverting the label encoding to what the value actually was

In [109]:
# First dealing with the new data and establishing the new dataframes

temp_columns = []
for i in actual_data.columns:
    if not i == "CONTRACTOR_cat":
        temp_columns.append(i)

end_result = pd.DataFrame(data = Xtrans, columns = temp_columns)
contractors = pd.DataFrame(data = y, columns = ["CONTRACTOR_cat"])

In [110]:
# After inserting the new data we want to combine
end_result = pd.concat([end_result, contractors], axis = 1, sort=False)

In [111]:
end_result

,UWI,UWI_SIDETRACK,REPORT_START_DATE_YEAR,FRAC_GRADIENT,BREAKDOWN_PRESSURE,HYDROSTATIC_PRESSURE,TREAT_AVG_PRESSURE,TREAT_MAX_PRESSURE,TREAT_MIN_PRESSURE,PROPPANT_DESIGNED,...,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,CLOSURE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat,SITE_SUPERVISOR_cat,CONTRACTOR_cat
0,2.019001e+09,0.0,2019.0,1.0,6740.0,5576.0,9358.0,9765.0,7346.0,152125.0,...,8.0,0.0,0.0,0.0,1.0,0.0,1074.0,33.0,45.0,2.0
1,2.019001e+09,0.0,2018.0,1.0,5100.0,5724.0,7280.0,9347.0,5100.0,0.0,...,15.0,0.0,0.0,0.0,1.0,0.0,699.0,20.0,29.8,3.0
2,2.019000e+09,0.0,2017.0,1.0,4901.0,5087.0,4999.0,5384.0,6240.2,0.0,...,5.0,0.0,0.0,0.0,1.0,0.0,258.0,32.0,29.8,2.0
3,2.019001e+09,0.0,2019.0,1.0,5304.0,5623.0,9496.0,9602.0,7563.6,130000.0,...,8.0,0.0,0.0,0.0,1.0,0.0,1419.0,24.0,11.0,3.0
4,2.019001e+09,0.0,2019.0,1.0,7340.0,5494.0,9523.0,9916.0,7618.0,366938.0,...,8.0,0.0,0.0,0.0,1.0,0.0,1179.0,24.0,32.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,2.019001e+09,0.0,2019.0,1.0,6954.0,4844.0,8928.0,10377.0,2964.0,192000.0,...,8.0,0.0,0.0,0.0,1.0,0.0,320.0,33.0,59.0,3.0
1650,2.019001e+09,0.0,2019.0,1.0,4773.0,4837.0,8697.0,9622.0,2964.0,192000.0,...,8.0,0.0,0.0,0.0,1.0,0.0,1375.0,33.0,47.0,3.0
1651,2.019001e+09,0.0,2019.0,1.0,5336.0,4829.0,8929.0,10049.0,2964.0,192000.0,...,8.0,0.0,0.0,0.0,1.0,0.0,479.0,33.0,47.0,3.0
1652,2.019001e+09,0.0,2019.0,1.0,6099.0,4818.0,9094.0,9721.0,2964.0,192000.0,...,8.0,0.0,0.0,0.0,1.0,0.0,1252.0,33.0,47.0,3.0


In [112]:
for i in end_result.columns:
    print(i)

UWI
UWI_SIDETRACK
REPORT_START_DATE_YEAR
FRAC_GRADIENT
BREAKDOWN_PRESSURE
HYDROSTATIC_PRESSURE
TREAT_AVG_PRESSURE
TREAT_MAX_PRESSURE
TREAT_MIN_PRESSURE
PROPPANT_DESIGNED
PROPPANT_IN_FORMATION
PROPPANT_IN_WELLBORE
AVG_HYDRAULIC_POWER
MAX_PUMP_POWER
MAX_PUMP_RATING
TREAT_AVG_RATE
TREAT_MAX_RATE
TREAT_MIN_RATE
POST_INSTANT_SHUT_IN_PRESSURE
INITIAL_SHUT_IN_PRESSURE
PRE_INSTANT_SHUT_IN_PRESSURE
STG_NUMBER
STG_TOP_DEPTH
STG_BOTTOM_DEPTH
STG_ELEMENT_TOP_DEPTH
STG_ELEMENT_BOTTOM_DEPTH
STG_INITIAL_PUMPING_PRESSURE
STG_FINAL_PUMPING_PRESSURE
STG_BREAKDOWN_PRESSURE
STG_CLEAN_VOLUME_PUMPED
STG_SLURRY_VOLUME_PUMPED
STG_VOLUME_RECOVERED
PUMPSONLINENO
USERNUM1
USERNUM3
DURPUMP
TOP_DEPTH_TVD
BOTTOM_DEPTH_TVD
PROPOSED
Rig_Duration (days)
Stg_Duration (hrs)
Stage_Length (meters)
Prod_Per_Stage
Errors_Per_Stage
JOB_START_Y
JOB_START_M
JOB_START_D
JOB_START_H
JOB_START_MIN
JOB_END_Y
JOB_END_M
JOB_END_D
JOB_END_H
JOB_END_MIN
PHASE_START_Y
PHASE_START_M
PHASE_START_D
PHASE_START_H
PHASE_START_MIN
PHASE_END_

In [113]:
# Here we are constructing a poor-mans label encoder translater dictionary to
# be able to translate the label codes to their respective labels

temp_map = {
}

for column in init_columns:
    unique_pair = [[], []]
    cat_name = column + "_cat"
    
    for index in range(0, len(all_obs)):
        label_data = all_obs.loc[index, column]
        num_data = all_obs.loc[index, cat_name]
        
        if not label_data in unique_pair[0]:
            unique_pair[0].append(label_data)
            unique_pair[1].append(num_data)
    temp_map[column] = unique_pair
        

In [114]:
temp_map

{'#WELL_NAME': [['WELL - 2019000699',
   'WELL - 2019000665',
   'WELL - 2019000347',
   'WELL - 2019001412',
   'WELL - 2019001390',
   'WELL - 2019001382',
   'WELL - 2019000343',
   'WELL - 2019000668',
   'WELL - 2019000633',
   'WELL - 2019001190',
   'WELL - 2019000642',
   'WELL - 2019001396',
   'WELL - 2019001290',
   'WELL - 2019000708',
   'WELL - 2019000730',
   'WELL - 2019000651',
   'WELL - 2019001299',
   'WELL - 2019000659',
   'WELL - 2019001277',
   'WELL - 2019000736',
   'WELL - 2019001293',
   'WELL - 2019000359',
   'WELL - 2019001300',
   'WELL - 2019001310',
   'WELL - 2019000650',
   'WELL - 2019001350',
   'WELL - 2019001366',
   'WELL - 2019001340',
   'WELL - 2019000660',
   'WELL - 2019001301',
   'WELL - 2019000715',
   'WELL - 2019000733',
   'WELL - 2019000735',
   'WELL - 2019000278',
   'WELL - 2019000640',
   'WELL - 2019000669',
   'WELL - 2019000687',
   'WELL - 2019000667',
   'WELL - 2019000225',
   'WELL - 2019000714',
   'WELL - 2019000655',
  

In [115]:
# Using that poor mans translater dictionary, we will try to match the the given
# label code and match it to its respective label
# Will return the "key" to the correct label

def find_closest(cat_code, cat_map, col):
    if cat_code in cat_map[col][1]:
        return cat_map[col][1].index(cat_code, 0, len(cat_map[col][1]))
    else:
        found = False
        before_code = cat_code - 1
        after_code = cat_code + 1
        while not found:
            if before_code in cat_map[col][1]:
                return cat_map[col][1].index(before_code, 0, len(cat_map[col][1]))
            
            if after_code in cat_map[col][1]:
                return cat_map[col][1].index(after_code, 0, len(cat_map[col][1]))
            
            before_code -= 1
            after_code += 1

In [116]:
# Now we actually construct the correct labels for what the imputation gave us

for column in init_columns:
    cat_column = column + "_cat"
    
    end_result[column] = np.nan
    
    for index in range(len(end_result)):
        cat_code = math.floor(end_result.loc[index, cat_column])
        
        label_index = find_closest(cat_code, temp_map, column)
        
        end_result.loc[index, column] = temp_map[column][0][label_index]
        
    print("Finished encoding for", column)
print("Finished everything")
        

Finished encoding for #WELL_NAME
Finished encoding for CURRENT_STATUS
Finished encoding for JOB_CATEGORY
Finished encoding for JOB_STATUS
Finished encoding for LIKE_KIND
Finished encoding for PROJECT_TYPE
Finished encoding for PHASE
Finished encoding for JOB_OP_CODE
Finished encoding for RIG_ID
Finished encoding for RIG_TYPE
Finished encoding for CONTRACTOR
Finished encoding for STIMULATION_TREAT_TYPE
Finished encoding for STIM_TREAT_COMPANY
Finished encoding for STIM_TREAT_SUPERVISOR
Finished encoding for DIVERSION_COMPANY
Finished encoding for DIVERSION_METHOD
Finished encoding for DELIVERY_MODE
Finished encoding for BOTTOM_HOLE_PRESSURE_METHOD
Finished encoding for CLOSURE_PRESSURE_METHOD
Finished encoding for STIMULATION_RESULT
Finished encoding for STIMULATION_RESULT_DETAIL
Finished encoding for STIMTREAT_ID
Finished encoding for FLUID_NAME
Finished encoding for SITE_SUPERVISOR
Finished everything


In [117]:
# Now after imputation, we get to see the results
count = 0
for i in end_result.columns:
# count number of rows with missing values
    n_miss = end_result[i].isnull().sum()
    perc = n_miss / end_result.shape[0] * 100
    print('> %2d %28s, Missing: %d (%.1f%%)' % (count, i, n_miss, perc))
    count += 1

>  0                          UWI, Missing: 0 (0.0%)
>  1                UWI_SIDETRACK, Missing: 0 (0.0%)
>  2       REPORT_START_DATE_YEAR, Missing: 0 (0.0%)
>  3                FRAC_GRADIENT, Missing: 0 (0.0%)
>  4           BREAKDOWN_PRESSURE, Missing: 0 (0.0%)
>  5         HYDROSTATIC_PRESSURE, Missing: 0 (0.0%)
>  6           TREAT_AVG_PRESSURE, Missing: 0 (0.0%)
>  7           TREAT_MAX_PRESSURE, Missing: 0 (0.0%)
>  8           TREAT_MIN_PRESSURE, Missing: 0 (0.0%)
>  9            PROPPANT_DESIGNED, Missing: 0 (0.0%)
> 10        PROPPANT_IN_FORMATION, Missing: 0 (0.0%)
> 11         PROPPANT_IN_WELLBORE, Missing: 0 (0.0%)
> 12          AVG_HYDRAULIC_POWER, Missing: 0 (0.0%)
> 13               MAX_PUMP_POWER, Missing: 0 (0.0%)
> 14              MAX_PUMP_RATING, Missing: 0 (0.0%)
> 15               TREAT_AVG_RATE, Missing: 0 (0.0%)
> 16               TREAT_MAX_RATE, Missing: 0 (0.0%)
> 17               TREAT_MIN_RATE, Missing: 0 (0.0%)
> 18 POST_INSTANT_SHUT_IN_PRESSURE, Missing: 0

In [118]:
# Now to export to then use in our model building
end_result.to_excel(path + "RCOMPL_KNN_IMPUTED.xlsx", index = False)

## Stimulation Stage

So seeing how this is pretty much exactly the same so the comments will be sparse

In [144]:
# Load in data and make sure we have no null data in the contractor
data_pd =  pd.read_excel(path + "TO_TACC_STM_STG.xlsx")
dataframe = data_pd[data_pd['CONTRACTOR'].notnull()]

In [145]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17721 entries, 0 to 17720
Data columns (total 77 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   #WELL_NAME                     17721 non-null  object        
 1   CURRENT_STATUS                 17721 non-null  object        
 2   UWI                            17721 non-null  int64         
 3   UWI_SIDETRACK                  17721 non-null  int64         
 4   JOB_CATEGORY                   17721 non-null  object        
 5   JOB_START_DATE                 17721 non-null  object        
 6   JOB_END_DATE                   8108 non-null   object        
 7   JOB_STATUS                     14349 non-null  object        
 8   LIKE_KIND                      17636 non-null  object        
 9   PROJECT_TYPE                   17721 non-null  object        
 10  PHASE_START_DATE               17702 non-null  object        
 11  PHASE_END_DATE 

## Manipulating date

In [146]:
# Getting dates data to then turn into pd.datetime objects

dates = ['JOB_START_DATE', 
         'JOB_END_DATE', 
         'PHASE_START_DATE', 
         'PHASE_END_DATE',  
         'REPORT_START_DATE_YEAR', 
         'REPORT_START_DATE', 
         'REPORT_END_DATE', 
         'STIMULATION_TREAT_DATE']

for d in dates:
    dataframe[d] = pd.to_datetime(dataframe[d], dayfirst=True)

In [147]:
# collect the datetiem objects
temp = dataframe.loc[:, dataframe.dtypes=='datetime64[ns]']

In [148]:
temp

,JOB_START_DATE,JOB_END_DATE,PHASE_START_DATE,PHASE_END_DATE,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,REPORT_START_DATE_YEAR,REPORT_START_DATE,REPORT_END_DATE,STIMULATION_TREAT_DATE,STG_START_DATE,STG_END_DATE
0,2017-10-04 00:00:00,2018-04-26 09:30:00,2017-04-14 17:15:00,2017-08-19 18:00:00,2017-07-26 06:00:00,2017-08-18 18:00:00,1970-01-01 00:00:00.000002017,2017-07-26 06:00:00,2017-07-27 06:00:00,2017-07-27 02:31:00,2017-07-27 02:31:00,2017-07-27 04:31:00
1,2017-06-12 07:00:00,2018-06-08 18:00:00,2017-11-12 17:00:00,2018-09-02 09:00:00,2018-01-20 06:00:00,2018-09-02 09:00:00,1970-01-01 00:00:00.000002018,2018-01-23 06:00:00,2018-01-24 06:00:00,2018-01-23 20:33:00,2018-01-23 20:33:00,2018-01-23 21:22:00
2,2018-12-16 06:00:00,2019-12-10 10:00:00,2019-11-01 06:00:00,2019-06-02 07:00:00,2019-01-27 06:00:00,2019-03-02 18:00:00,1970-01-01 00:00:00.000002019,2019-01-28 06:00:00,2019-01-29 06:00:00,2019-01-28 08:05:00,2019-01-28 08:05:00,2019-01-28 10:05:00
3,2018-01-21 10:00:00,2018-09-10 07:00:00,2018-01-27 06:00:00,2018-02-23 18:00:00,2018-02-14 08:00:00,2018-02-23 18:00:00,1970-01-01 00:00:00.000002018,2018-02-19 06:00:00,2018-02-20 06:00:00,2018-02-19 14:40:00,2018-02-19 14:40:00,2018-02-20 00:19:00
4,2017-01-08 06:00:00,NaT,2017-08-25 06:00:00,2017-05-10 06:00:00,2017-08-09 06:00:00,2017-02-10 06:00:00,1970-01-01 00:00:00.000002017,2017-11-09 06:00:00,2017-12-09 06:00:00,2017-11-09 13:35:00,2017-11-09 13:35:00,2017-11-09 15:33:00
...,...,...,...,...,...,...,...,...,...,...,...,...
17716,2019-07-19 06:00:00,NaT,2019-07-23 06:00:00,2019-06-10 17:00:00,2019-08-20 06:00:00,2019-04-09 06:00:00,1970-01-01 00:00:00.000002019,2019-08-25 06:00:00,2019-08-26 06:00:00,2019-08-25 18:59:00,2019-08-25 18:59:00,2019-08-25 20:42:00
17717,2018-08-30 09:00:00,2019-03-05 13:30:00,2018-04-09 16:00:00,2018-11-23 18:00:00,2018-08-10 06:00:00,2018-06-11 06:00:00,1970-01-01 00:00:00.000002018,2018-10-19 06:00:00,2018-10-20 06:00:00,2018-10-20 01:59:00,2018-10-20 01:59:00,2018-10-20 03:47:00
17718,2019-07-19 06:00:00,NaT,2019-07-23 06:00:00,2019-06-10 17:00:00,2019-08-20 06:00:00,2019-04-09 06:00:00,1970-01-01 00:00:00.000002019,2019-08-23 06:00:00,2019-08-24 06:00:00,2019-08-23 08:14:00,2019-08-23 08:14:00,2019-08-23 10:46:00
17719,2018-08-30 09:00:00,2019-03-05 13:30:00,2018-04-09 16:00:00,2018-11-23 18:00:00,2018-08-10 06:00:00,2018-06-11 06:00:00,1970-01-01 00:00:00.000002018,2018-10-30 06:00:00,2018-10-31 06:00:00,2018-10-31 01:47:00,2018-10-31 01:47:00,2018-10-31 03:37:00


In [149]:
date_features = temp.columns

In [150]:
# separate the month, year ... from the dates
for i in date_features:
    split_date(i);
    print("Finished", i)

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_l

Finished JOB_START_DATE
Finished JOB_END_DATE
Finished PHASE_START_DATE
Finished PHASE_END_DATE
Finished RIG_ACCEPT_DATE
Finished RIG_RELEASE_DATE
Finished REPORT_START_DATE_YEAR
Finished REPORT_START_DATE
Finished REPORT_END_DATE
Finished STIMULATION_TREAT_DATE
Finished STG_START_DATE
Finished STG_END_DATE


In [151]:
temp

,JOB_START_DATE,JOB_END_DATE,PHASE_START_DATE,PHASE_END_DATE,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,REPORT_START_DATE_YEAR,REPORT_START_DATE,REPORT_END_DATE,STIMULATION_TREAT_DATE,...,STG_START_Y,STG_START_M,STG_START_D,STG_START_H,STG_START_MIN,STG_END_Y,STG_END_M,STG_END_D,STG_END_H,STG_END_MIN
0,2017-10-04 00:00:00,2018-04-26 09:30:00,2017-04-14 17:15:00,2017-08-19 18:00:00,2017-07-26 06:00:00,2017-08-18 18:00:00,1970-01-01 00:00:00.000002017,2017-07-26 06:00:00,2017-07-27 06:00:00,2017-07-27 02:31:00,...,2017.0,7.0,27.0,2.0,31.0,2017.0,7.0,27.0,4.0,31.0
1,2017-06-12 07:00:00,2018-06-08 18:00:00,2017-11-12 17:00:00,2018-09-02 09:00:00,2018-01-20 06:00:00,2018-09-02 09:00:00,1970-01-01 00:00:00.000002018,2018-01-23 06:00:00,2018-01-24 06:00:00,2018-01-23 20:33:00,...,2018.0,1.0,23.0,20.0,33.0,2018.0,1.0,23.0,21.0,22.0
2,2018-12-16 06:00:00,2019-12-10 10:00:00,2019-11-01 06:00:00,2019-06-02 07:00:00,2019-01-27 06:00:00,2019-03-02 18:00:00,1970-01-01 00:00:00.000002019,2019-01-28 06:00:00,2019-01-29 06:00:00,2019-01-28 08:05:00,...,2019.0,1.0,28.0,8.0,5.0,2019.0,1.0,28.0,10.0,5.0
3,2018-01-21 10:00:00,2018-09-10 07:00:00,2018-01-27 06:00:00,2018-02-23 18:00:00,2018-02-14 08:00:00,2018-02-23 18:00:00,1970-01-01 00:00:00.000002018,2018-02-19 06:00:00,2018-02-20 06:00:00,2018-02-19 14:40:00,...,2018.0,2.0,19.0,14.0,40.0,2018.0,2.0,20.0,0.0,19.0
4,2017-01-08 06:00:00,NaT,2017-08-25 06:00:00,2017-05-10 06:00:00,2017-08-09 06:00:00,2017-02-10 06:00:00,1970-01-01 00:00:00.000002017,2017-11-09 06:00:00,2017-12-09 06:00:00,2017-11-09 13:35:00,...,2017.0,11.0,9.0,13.0,35.0,2017.0,11.0,9.0,15.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,2019-07-19 06:00:00,NaT,2019-07-23 06:00:00,2019-06-10 17:00:00,2019-08-20 06:00:00,2019-04-09 06:00:00,1970-01-01 00:00:00.000002019,2019-08-25 06:00:00,2019-08-26 06:00:00,2019-08-25 18:59:00,...,2019.0,8.0,25.0,18.0,59.0,2019.0,8.0,25.0,20.0,42.0
17717,2018-08-30 09:00:00,2019-03-05 13:30:00,2018-04-09 16:00:00,2018-11-23 18:00:00,2018-08-10 06:00:00,2018-06-11 06:00:00,1970-01-01 00:00:00.000002018,2018-10-19 06:00:00,2018-10-20 06:00:00,2018-10-20 01:59:00,...,2018.0,10.0,20.0,1.0,59.0,2018.0,10.0,20.0,3.0,47.0
17718,2019-07-19 06:00:00,NaT,2019-07-23 06:00:00,2019-06-10 17:00:00,2019-08-20 06:00:00,2019-04-09 06:00:00,1970-01-01 00:00:00.000002019,2019-08-23 06:00:00,2019-08-24 06:00:00,2019-08-23 08:14:00,...,2019.0,8.0,23.0,8.0,14.0,2019.0,8.0,23.0,10.0,46.0
17719,2018-08-30 09:00:00,2019-03-05 13:30:00,2018-04-09 16:00:00,2018-11-23 18:00:00,2018-08-10 06:00:00,2018-06-11 06:00:00,1970-01-01 00:00:00.000002018,2018-10-30 06:00:00,2018-10-31 06:00:00,2018-10-31 01:47:00,...,2018.0,10.0,31.0,1.0,47.0,2018.0,10.0,31.0,3.0,37.0


In [152]:
# Combine with the overall data
dataframe = pd.concat([dataframe, temp], axis = 1, sort=False)

In [153]:
# Remove the original date columns
dataframe = dataframe.drop(date_features, axis = 1)

In [154]:
dataframe

,#WELL_NAME,CURRENT_STATUS,UWI,UWI_SIDETRACK,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,...,STG_START_Y,STG_START_M,STG_START_D,STG_START_H,STG_START_MIN,STG_END_Y,STG_END_M,STG_END_D,STG_END_H,STG_END_MIN
0,WELL - 2019000507,Producing,2019000507,0,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2017.0,7.0,27.0,2.0,31.0,2017.0,7.0,27.0,4.0,31.0
1,WELL - 2019000477,Producing,2019000477,0,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2018.0,1.0,23.0,20.0,33.0,2018.0,1.0,23.0,21.0,22.0
2,WELL - 2019000699,Producing,2019000699,0,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2019.0,1.0,28.0,8.0,5.0,2019.0,1.0,28.0,10.0,5.0
3,WELL - 2019000665,Producing,2019000665,0,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,...,2018.0,2.0,19.0,14.0,40.0,2018.0,2.0,20.0,0.0,19.0
4,WELL - 2019000466,Producing,2019000466,0,COMPLETIONS,PRODUCING,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2017.0,11.0,9.0,13.0,35.0,2017.0,11.0,9.0,15.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,WELL - 2019000803,Suspended (I),2019000803,99,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2019.0,8.0,25.0,18.0,59.0,2019.0,8.0,25.0,20.0,42.0
17717,WELL - 2019000078,Suspended (I),2019000078,0,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2018.0,10.0,20.0,1.0,59.0,2018.0,10.0,20.0,3.0,47.0
17718,WELL - 2019000803,Suspended (I),2019000803,99,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2019.0,8.0,23.0,8.0,14.0,2019.0,8.0,23.0,10.0,46.0
17719,WELL - 2019000078,Suspended (I),2019000078,0,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2018.0,10.0,31.0,1.0,47.0,2018.0,10.0,31.0,3.0,37.0


## Label Encoding

In [155]:
# Collecting all the objects to then use label encoding
all_obs = dataframe.loc[:, dataframe.dtypes==object]

In [156]:
all_obs

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,RIG_ID,RIG_TYPE,...,STIM_TREAT_COMPANY,STIM_TREAT_SUPERVISOR,DIVERSION_COMPANY,DIVERSION_METHOD,DELIVERY_MODE,BOTTOM_HOLE_PRESSURE_METHOD,STIMULATION_RESULT,STIMULATION_RESULT_DETAIL,STIMTREAT_ID,FLUID_NAME
0,WELL - 2019000507,Producing,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000002,ROBERT,DIVERSION_COMPANY - 2019000003,Ball,Casing,NaN,Success,According to Plan,6652C6217F11405488E785A3D84CFD56,25# XL
1,WELL - 2019000477,Producing,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000004,LUPE,DIVERSION_COMPANY - 2019000003,Ball,Casing,Calculated,Success,According to Plan,9808C6BE9B2746A9A63D58EEB3885939,25# GUAR BORATE
2,WELL - 2019000699,Producing,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000004,NaN,DIVERSION_COMPANY - 2019000007,Frac Plug,Casing,NaN,Success,According to Plan,A6066AE1E85F4B1DB15D9940B5FBBEBF,Slickwater
3,WELL - 2019000665,Producing,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000005,JEFF,DIVERSION_COMPANY - 2019000037,SILICA FLOUR,Casing,NaN,Success,According to Plan,6ACEA3AEC54E415B8251B86897A229F3,FRESH WATER
4,WELL - 2019000466,Producing,COMPLETIONS,PRODUCING,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000002,ERNIE,DIVERSION_COMPANY - 2019000003,Ball,Casing,NaN,Success,According to Plan,0459426A0021476CADD000E29A193E8D,Slick Water
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,WELL - 2019000803,Suspended (I),COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000002,NaN,NaN,NaN,Casing,NaN,Success,According to Plan,565A1643FA9140459F2DF447D84C47D7,X-link gel
17717,WELL - 2019000078,Suspended (I),COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000004,NaN,NaN,NaN,Casing,NaN,Success,According to Plan,69DEBE4C29C3464A856BFF7A0533DF1A,X-link gel
17718,WELL - 2019000803,Suspended (I),COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000002,NaN,NaN,NaN,Casing,NaN,Success,According to Plan,1E7352090D724637A0CCDCD80C5E716C,X-link gel
17719,WELL - 2019000078,Suspended (I),COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,STIM_TREAT_COMPANY - 2019000004,NaN,NaN,NaN,Casing,NaN,NaN,NaN,282547570616450B85C55AFAB23D1082,X-link gel


In [157]:
# will be using pandas category codes as a makeshift label encoding

init_columns = all_obs.columns

for c in init_columns:
    all_obs[c] = all_obs[c].astype('category')
    cat_label = c + "_cat"
    all_obs[cat_label] = all_obs[c].cat.codes

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [158]:
# Taking those cat codes and replacing the -1s with nan
for c in init_columns:
    cat_label = c + "_cat"
    all_obs[cat_label] = all_obs[cat_label].replace([-1], np.nan)

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [159]:
all_obs

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,RIG_ID,RIG_TYPE,...,STIM_TREAT_COMPANY_cat,STIM_TREAT_SUPERVISOR_cat,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat
0,WELL - 2019000507,Producing,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,0.0,69.0,0.0,4.0,1.0,NaN,4.0,5.0,7160,27.0
1,WELL - 2019000477,Producing,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,2.0,52.0,0.0,4.0,1.0,0.0,4.0,5.0,10650,30.0
2,WELL - 2019000699,Producing,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,2.0,NaN,2.0,11.0,1.0,NaN,4.0,5.0,11618,67.0
3,WELL - 2019000665,Producing,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,RIG_ID - 2019000098,FRAC EQUIP,...,3.0,42.0,21.0,19.0,1.0,NaN,4.0,5.0,7514,48.0
4,WELL - 2019000466,Producing,COMPLETIONS,PRODUCING,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,0.0,33.0,0.0,4.0,1.0,NaN,4.0,5.0,297,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,WELL - 2019000803,Suspended (I),COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,0.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,5988,75.0
17717,WELL - 2019000078,Suspended (I),COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,2.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,7438,75.0
17718,WELL - 2019000803,Suspended (I),COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000021,FRAC EQUIP,...,0.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,2085,75.0
17719,WELL - 2019000078,Suspended (I),COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,RIG_ID - 2019000082,FRAC EQUIP,...,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2741,75.0


In [160]:
# Drop the initial label columns
temp_df = all_obs.drop(init_columns, axis=1)

In [161]:
temp_df

,#WELL_NAME_cat,CURRENT_STATUS_cat,JOB_CATEGORY_cat,JOB_STATUS_cat,LIKE_KIND_cat,PROJECT_TYPE_cat,PHASE_cat,JOB_OP_CODE_cat,RIG_ID_cat,RIG_TYPE_cat,...,STIM_TREAT_COMPANY_cat,STIM_TREAT_SUPERVISOR_cat,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat
0,182,1,0,0.0,2.0,0,0,0,0,1.0,...,0.0,69.0,0.0,4.0,1.0,NaN,4.0,5.0,7160,27.0
1,153,1,0,NaN,2.0,0,0,0,2,1.0,...,2.0,52.0,0.0,4.0,1.0,0.0,4.0,5.0,10650,30.0
2,217,1,0,0.0,1.0,1,0,0,2,1.0,...,2.0,NaN,2.0,11.0,1.0,NaN,4.0,5.0,11618,67.0
3,209,1,0,0.0,3.0,1,0,0,5,1.0,...,3.0,42.0,21.0,19.0,1.0,NaN,4.0,5.0,7514,48.0
4,142,1,0,2.0,2.0,0,0,0,0,1.0,...,0.0,33.0,0.0,4.0,1.0,NaN,4.0,5.0,297,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,241,2,0,NaN,2.0,0,0,0,0,1.0,...,0.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,5988,75.0
17717,39,2,0,0.0,2.0,0,0,0,2,1.0,...,2.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,7438,75.0
17718,241,2,0,NaN,2.0,0,0,0,0,1.0,...,0.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,2085,75.0
17719,39,2,0,0.0,2.0,0,0,0,2,1.0,...,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2741,75.0


In [162]:
# Combine the cat codes with temp data
data_with_labels = pd.concat([dataframe, temp_df], axis=1, sort = False)

In [163]:
data_with_labels

,#WELL_NAME,CURRENT_STATUS,UWI,UWI_SIDETRACK,JOB_CATEGORY,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE,JOB_OP_CODE,...,STIM_TREAT_COMPANY_cat,STIM_TREAT_SUPERVISOR_cat,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat
0,WELL - 2019000507,Producing,2019000507,0,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,0.0,69.0,0.0,4.0,1.0,NaN,4.0,5.0,7160,27.0
1,WELL - 2019000477,Producing,2019000477,0,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2.0,52.0,0.0,4.0,1.0,0.0,4.0,5.0,10650,30.0
2,WELL - 2019000699,Producing,2019000699,0,COMPLETIONS,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,COMPZN,STIM,...,2.0,NaN,2.0,11.0,1.0,NaN,4.0,5.0,11618,67.0
3,WELL - 2019000665,Producing,2019000665,0,COMPLETIONS,COMPLETED,OCM - PP TUB,RECOMPLETION,COMPZN,STIM,...,3.0,42.0,21.0,19.0,1.0,NaN,4.0,5.0,7514,48.0
4,WELL - 2019000466,Producing,2019000466,0,COMPLETIONS,PRODUCING,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,0.0,33.0,0.0,4.0,1.0,NaN,4.0,5.0,297,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,WELL - 2019000803,Suspended (I),2019000803,99,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,0.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,5988,75.0
17717,WELL - 2019000078,Suspended (I),2019000078,0,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,7438,75.0
17718,WELL - 2019000803,Suspended (I),2019000803,99,COMPLETIONS,NaN,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,0.0,NaN,NaN,NaN,1.0,NaN,4.0,5.0,2085,75.0
17719,WELL - 2019000078,Suspended (I),2019000078,0,COMPLETIONS,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,COMPZN,STIM,...,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2741,75.0


In [164]:
# Remove the columns for the actual main data
actual_data = data_with_labels.drop(init_columns, axis=1)

In [165]:
# seeing how many missing entries we have initialy

count = 0
for i in actual_data.columns:
# count number of rows with missing values
    n_miss = actual_data[i].isnull().sum()
    perc = n_miss / actual_data.shape[0] * 100
    print('> %2d %28s, Missing: %d (%.1f%%)' % (count, i, n_miss, perc))
    count += 1

>  0                          UWI, Missing: 0 (0.0%)
>  1                UWI_SIDETRACK, Missing: 0 (0.0%)
>  2                FRAC_GRADIENT, Missing: 453 (2.6%)
>  3           BREAKDOWN_PRESSURE, Missing: 316 (1.8%)
>  4         HYDROSTATIC_PRESSURE, Missing: 153 (0.9%)
>  5           TREAT_AVG_PRESSURE, Missing: 6 (0.0%)
>  6           TREAT_MAX_PRESSURE, Missing: 5 (0.0%)
>  7           TREAT_MIN_PRESSURE, Missing: 9798 (55.3%)
>  8            PROPPANT_DESIGNED, Missing: 3 (0.0%)
>  9        PROPPANT_IN_FORMATION, Missing: 2 (0.0%)
> 10         PROPPANT_IN_WELLBORE, Missing: 7268 (41.0%)
> 11          AVG_HYDRAULIC_POWER, Missing: 90 (0.5%)
> 12               MAX_PUMP_POWER, Missing: 132 (0.7%)
> 13              MAX_PUMP_RATING, Missing: 2919 (16.5%)
> 14               TREAT_AVG_RATE, Missing: 6 (0.0%)
> 15               TREAT_MAX_RATE, Missing: 5 (0.0%)
> 16               TREAT_MIN_RATE, Missing: 26 (0.1%)
> 17 POST_INSTANT_SHUT_IN_PRESSURE, Missing: 854 (4.8%)
> 18     INITIAL_SHUT

## KNN Imputation

In [178]:
# taking the data that we are working with and using Contractor as our y

data = actual_data.values
ix = [i for i in range(data.shape[1]) if i != 113]
X, y = data[:, ix], data[:, 113]

In [179]:
# define imputer
imputer = KNNImputer()
# fit on the dataset
imputer.fit(X)

KNNImputer()

In [180]:
# Transform our data so that it has no nans
Xtrans = imputer.transform(X)

In [181]:
print('missing: %d' % sum(np.isnan(Xtrans).flatten()))

missing: 0


In [182]:
# Starting the process to reverse the label encoding

temp_columns = []
for i in actual_data.columns:
    if not i == "CONTRACTOR_cat":
        temp_columns.append(i)

end_result = pd.DataFrame(data = Xtrans, columns = temp_columns)
contractors = pd.DataFrame(data = y, columns = ["CONTRACTOR_cat"])

In [183]:
end_result = pd.concat([end_result, contractors], axis = 1, sort=False)

In [184]:
end_result

,UWI,UWI_SIDETRACK,FRAC_GRADIENT,BREAKDOWN_PRESSURE,HYDROSTATIC_PRESSURE,TREAT_AVG_PRESSURE,TREAT_MAX_PRESSURE,TREAT_MIN_PRESSURE,PROPPANT_DESIGNED,PROPPANT_IN_FORMATION,...,STIM_TREAT_SUPERVISOR_cat,DIVERSION_COMPANY_cat,DIVERSION_METHOD_cat,DELIVERY_MODE_cat,BOTTOM_HOLE_PRESSURE_METHOD_cat,STIMULATION_RESULT_cat,STIMULATION_RESULT_DETAIL_cat,STIMTREAT_ID_cat,FLUID_NAME_cat,CONTRACTOR_cat
0,2.019001e+09,0.0,1.0,6547.0,4984.0,9919.0,10176.0,6550.0,104000.0,104060.0,...,69.0,0.0,4.0,1.0,0.0,4.0,5.0,7160.0,27.0,0.0
1,2.019000e+09,0.0,1.0,7348.0,5031.0,10027.0,10525.0,7348.0,104000.0,104000.0,...,52.0,0.0,4.0,1.0,0.0,4.0,5.0,10650.0,30.0,2.0
2,2.019001e+09,0.0,1.0,6740.0,5576.0,9358.0,9765.0,7346.0,152125.0,152125.0,...,30.0,2.0,11.0,1.0,0.0,4.0,5.0,11618.0,67.0,2.0
3,2.019001e+09,0.0,1.0,5100.0,5724.0,7280.0,9347.0,5100.0,0.0,0.0,...,42.0,21.0,19.0,1.0,0.0,4.0,5.0,7514.0,48.0,3.0
4,2.019000e+09,0.0,1.0,6026.0,4696.0,8298.0,9349.0,6026.0,104000.0,104780.0,...,33.0,0.0,4.0,1.0,0.0,4.0,5.0,297.0,66.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,2.019001e+09,99.0,1.0,7360.0,5695.0,11086.0,11211.0,8190.0,420000.0,420000.0,...,23.2,5.0,11.0,1.0,0.0,4.0,5.0,5988.0,75.0,0.0
17717,2.019000e+09,0.0,1.0,6913.0,5154.0,9695.0,10136.0,4428.0,420000.0,420000.0,...,73.4,0.0,1.6,1.0,0.0,4.0,5.0,7438.0,75.0,2.0
17718,2.019001e+09,99.0,1.0,8132.0,5714.0,10778.0,11080.0,8190.0,420000.0,420000.0,...,48.2,4.0,11.0,1.0,0.0,4.0,5.0,2085.0,75.0,0.0
17719,2.019000e+09,0.0,1.0,7366.0,5137.0,10098.0,10337.0,7179.0,420000.0,422369.0,...,64.8,0.0,2.8,1.0,0.0,4.0,5.0,2741.0,75.0,2.0


In [185]:
for i in end_result.columns:
    print(i)

UWI
UWI_SIDETRACK
FRAC_GRADIENT
BREAKDOWN_PRESSURE
HYDROSTATIC_PRESSURE
TREAT_AVG_PRESSURE
TREAT_MAX_PRESSURE
TREAT_MIN_PRESSURE
PROPPANT_DESIGNED
PROPPANT_IN_FORMATION
PROPPANT_IN_WELLBORE
AVG_HYDRAULIC_POWER
MAX_PUMP_POWER
MAX_PUMP_RATING
TREAT_AVG_RATE
TREAT_MAX_RATE
TREAT_MIN_RATE
POST_INSTANT_SHUT_IN_PRESSURE
INITIAL_SHUT_IN_PRESSURE
PRE_INSTANT_SHUT_IN_PRESSURE
STG_NUMBER
STG_TOP_DEPTH
STG_BOTTOM_DEPTH
STG_ELEMENT_TOP_DEPTH
STG_ELEMENT_BOTTOM_DEPTH
STG_INITIAL_PUMPING_PRESSURE
STG_FINAL_PUMPING_PRESSURE
STG_BREAKDOWN_PRESSURE
STG_CLEAN_VOLUME_PUMPED
STG_SLURRY_VOLUME_PUMPED
STG_VOLUME_RECOVERED
PUMPSONLINENO
USERNUM1
USERNUM3
DURPUMP
TOP_DEPTH_TVD
BOTTOM_DEPTH_TVD
PROPOSED
Rig_Duration (days)
Stg_Duration (hrs)
Stage_Length (meters)
Prod_Per_Stage
Errors_Per_Stage
JOB_START_Y
JOB_START_M
JOB_START_D
JOB_START_H
JOB_START_MIN
JOB_END_Y
JOB_END_M
JOB_END_D
JOB_END_H
JOB_END_MIN
PHASE_START_Y
PHASE_START_M
PHASE_START_D
PHASE_START_H
PHASE_START_MIN
PHASE_END_Y
PHASE_END_M
PHASE_END

In [186]:
temp_map = {
}

for column in init_columns:
    unique_pair = [[], []]
    cat_name = column + "_cat"
    
    for index in range(0, len(all_obs)):
        label_data = all_obs.loc[index, column]
        num_data = all_obs.loc[index, cat_name]
        
        if not label_data in unique_pair[0]:
            unique_pair[0].append(label_data)
            unique_pair[1].append(num_data)
    temp_map[column] = unique_pair

In [187]:
temp_map

{'#WELL_NAME': [['WELL - 2019000507',
   'WELL - 2019000477',
   'WELL - 2019000699',
   'WELL - 2019000665',
   'WELL - 2019000466',
   'WELL - 2019000382',
   'WELL - 2019000966',
   'WELL - 2019000927',
   'WELL - 2019000837',
   'WELL - 2019000857',
   'WELL - 2019000457',
   'WELL - 2019000347',
   'WELL - 2019000814',
   'WELL - 2019000819',
   'WELL - 2019000406',
   'WELL - 2019000395',
   'WELL - 2019000922',
   'WELL - 2019000926',
   'WELL - 2019000011',
   'WELL - 2019000883',
   'WELL - 2019000456',
   'WELL - 2019000400',
   'WELL - 2019000459',
   'WELL - 2019000525',
   'WELL - 2019000482',
   'WELL - 2019000451',
   'WELL - 2019000848',
   'WELL - 2019000470',
   'WELL - 2019000900',
   'WELL - 2019000418',
   'WELL - 2019000514',
   'WELL - 2019000455',
   'WELL - 2019001412',
   'WELL - 2019000806',
   'WELL - 2019000436',
   'WELL - 2019000890',
   'WELL - 2019000909',
   'WELL - 2019000850',
   'WELL - 2019000432',
   'WELL - 2019000881',
   'WELL - 2019000005',
  

In [188]:
def find_closest(cat_code, cat_map, col):
    counter = 30000
    
    
    if cat_code in cat_map[col][1]:
        return cat_map[col][1].index(cat_code, 0, len(cat_map[col][1]))
    else:
        found = False
        before_code = cat_code - 1
        after_code = cat_code + 1
        while not found:
            if before_code in cat_map[col][1]:
                return cat_map[col][1].index(before_code, 0, len(cat_map[col][1]))
            
            if after_code in cat_map[col][1]:
                return cat_map[col][1].index(after_code, 0, len(cat_map[col][1]))
            
            if counter == 0:
                print("Tripped counter")
                return 0
            
            
            before_code -= 1
            after_code += 1
            counter -= 1

In [ ]:
for column in init_columns:
    cat_column = column + "_cat"
    
    end_result[column] = np.nan
    
    for index in range(len(end_result)):
        cat_code = math.floor(end_result.loc[index, cat_column])
        
        label_index = find_closest(cat_code, temp_map, column)
        
        end_result.loc[index, column] = temp_map[column][0][label_index]
        
    print("Finished encoding for", column)
print("Finished everything")
        

Finished encoding for #WELL_NAME
Finished encoding for CURRENT_STATUS
Finished encoding for JOB_CATEGORY
Finished encoding for JOB_STATUS
Finished encoding for LIKE_KIND
Finished encoding for PROJECT_TYPE
Finished encoding for PHASE
Finished encoding for JOB_OP_CODE
Finished encoding for RIG_ID
Finished encoding for RIG_TYPE
Finished encoding for CONTRACTOR
Finished encoding for STIMULATION_TREAT_TYPE
Finished encoding for STIM_TREAT_COMPANY
Finished encoding for STIM_TREAT_SUPERVISOR
Finished encoding for DIVERSION_COMPANY
Finished encoding for DIVERSION_METHOD
Finished encoding for DELIVERY_MODE
Finished encoding for BOTTOM_HOLE_PRESSURE_METHOD
Finished encoding for STIMULATION_RESULT
Finished encoding for STIMULATION_RESULT_DETAIL


In [ ]:
end_result

In [ ]:
count = 0
for i in end_result.columns:
# count number of rows with missing values
    n_miss = end_result[i].isnull().sum()
    perc = n_miss / end_result.shape[0] * 100
    print('> %2d %28s, Missing: %d (%.1f%%)' % (count, i, n_miss, perc))
    count += 1

In [ ]:
end_result.to_excel(path + "STMSTG_KNN_IMPUTED.xlsx", index = False)